# Bilan hydrique à partir d'observations Météo-France quotidiennes qualifiées

## Reconstruction des données météorologiques pour une station de référence

### Définition des paramètres

In [1]:
# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
# Altitude (m)
REF_STATION_ALTITUDE = 50.

# Départements où chercher des stations
ID_DEPARTEMENTS = [35, 50]

NN_NOMBRE = None
# NN_RAYON_KM = 20.
NN_RAYON_KM = 35.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

### Lecture de la liste des stations

In [2]:
import meteofrance

# Météo-France API
METEOFRANCE_API = 'DPClim'

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'quotidienne'

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(METEOFRANCE_API, application_id=APPLICATION_ID)

In [3]:
import pandas as pd

LECTURE_LISTE_STATIONS = False

l_listes = []
for id_dep in ID_DEPARTEMENTS:
    filepath_liste_stations = meteofrance.get_filepath_liste_stations(
        client, frequence=METEOFRANCE_FREQUENCE, id_departement=id_dep)
    
    if LECTURE_LISTE_STATIONS:
        # Lecture de la liste des stations par département
        df_liste_stations_dep = pd.read_csv(
            filepath_liste_stations, index_col=client.id_station_label)
    else:
        # Demande de la liste des stations pour le département
        section = meteofrance.SECTION_LISTE_STATIONS
        params = {'id-departement': id_dep}
        response = meteofrance.demande(
            client, section, params=params, frequence=METEOFRANCE_FREQUENCE)
        df_liste_stations_dep = meteofrance.response_text_to_frame(
            client, response, index_col=client.id_station_label)

        # Sauvegarde de la liste des stations par département
        df_liste_stations_dep.to_csv(filepath_liste_stations)

    # Liste pour compilation
    l_listes.append(df_liste_stations_dep)

# Compilation
df_liste_stations_brute = pd.concat(l_listes, axis='index')

# Garder les stations valides seulement
df_liste_stations = meteofrance.filtrer_stations_valides(client, df_liste_stations_brute)

### Sélection des plus proches voisins de la station de référence

In [4]:
import geo

df_liste_stations_nn = geo.selection_stations_plus_proches(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

df_liste_stations_nn

,nom,typePoste,lon,lat,alt,distance
id,,,,,,
35044001,BROUALAN,2,-1.640833,48.485667,99,6
50410003,PONTORSON,1,-1.505167,48.585667,33,9
35224001,PLERGUER,2,-1.843667,48.524833,30,17
35110003,FEINS SA,1,-1.596833,48.326833,87,24
35225001,PLESDER,2,-1.924833,48.406833,56,27
35178001,MEZIERES-SUR-C.,2,-1.439000,48.308833,71,29
50531001,ST OVIN,2,-1.248667,48.682500,155,31
50218001,GRANVILLE,3,-1.613667,48.834500,37,33
35228001,DINARD,0,-2.076333,48.584833,65,34


### Obtention des données météorologiques pour les stations voisines

In [5]:
import bilan

# DATE_DEB_PERIODE = '2005-01-01T00:00:00Z'
DATE_DEB_PERIODE = '2022-01-01T00:00:00Z'
DATE_FIN_PERIODE = '2024-12-31T00:00:00Z'

LECTURE_DONNEE = False

# Ségmentation de la période par année
idx_dates_deb = pd.date_range(
    start=DATE_DEB_PERIODE, end=DATE_FIN_PERIODE, freq='YS-JAN')
idx_dates_fin = pd.date_range(
    start=DATE_DEB_PERIODE, end=DATE_FIN_PERIODE, freq='YE-DEC')
list_dates_deb = [d.isoformat().replace("+00:00", "Z") for d in idx_dates_deb]
list_dates_fin = [d.isoformat().replace("+00:00", "Z") for d in idx_dates_fin]

df_meteo = pd.DataFrame(dtype=float)
for date_deb, date_fin in zip(list_dates_deb, list_dates_fin):
    filepath_donnee_an = meteofrance.get_filepath_donnee_periode(
        client, REF_STATION_NAME, df_liste_stations_nn, date_deb, date_fin,
        frequence=METEOFRANCE_FREQUENCE)
    
    if LECTURE_DONNEE:
        # Lecture des données des stations pour la période
        df_meteo_an = pd.read_csv(
            filepath_donnee_an, parse_dates=[client.time_label],
            index_col=[client.id_station_donnee_label, client.time_label])
    else:
        # Demande des données des stations pour la période
        variables = [client.variables_labels[METEOFRANCE_FREQUENCE][k]
                     for k in bilan.VARIABLES_CALCUL_BILAN]
        df_meteo_an = meteofrance.compiler_telechargement_des_stations_periode(
            client, df_liste_stations_nn, date_deb, date_fin,
            frequence=METEOFRANCE_FREQUENCE)[variables]
    
        # Sauvegarde des données des stations pour la période par département
        df_meteo_an.to_csv(filepath_donnee_an)

    # Compilation des années
    df_meteo = pd.concat([df_meteo, df_meteo_an], axis='index')

HTTPError: 429 Client Error: Too Many Requests for url: https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier?id-cmde=791486242918

### Interpolation des données météorologiques à la station de référence

Les variables sont également renommées en utilisant des noms communs à l'ensemble de ces notebooks quelque soit l'API utilisée.

In [ ]:
LECTURE_DONNEE_REF = False

filepath_donnee_ref = meteofrance.get_filepath_donnee_periode(
    client, REF_STATION_NAME, df_liste_stations_nn, DATE_DEB_PERIODE, DATE_FIN_PERIODE,
    frequence=METEOFRANCE_FREQUENCE, ref=True)

if LECTURE_DONNEE_REF:
    # Lecture des données des stations pour la période
    df_meteo_ref = pd.read_csv(
        filepath_donnee_ref, parse_dates=[client.time_label],
        index_col=client.time_label)
else:
    # Demande des données des stations pour la période
    df_meteo_ref = geo.interpolation_inverse_distance_carre(
        df_meteo, df_liste_stations_nn['distance'])
    
    # Sauvegarde par département
    df_meteo_ref.to_csv(filepath_donnee_ref)

df_meteo_ref = meteofrance.renommer_variables(
    client, df_meteo_ref, METEOFRANCE_FREQUENCE)

### Conversion des unités

In [ ]:
df_meteo_ref_si = meteofrance.convertir_unites(
    client, df_meteo_ref)

## Estimation du bilan hydrique

### Définition des paramètres

In [ ]:
# RFU finale cible (mm)
RFU_CIBLE = None

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.1

# Choix de la texture
TEXTURE = 'Terres limoneuses'

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_RU_REMPLIE = 1.

# Besoin d'irrigation minimal à partir du quel irriguer (mm)
SEUIL_IRRIGATION = 1.

# Conversion de hauteur (mm) vers durée d'irrigation (min)
HAUTEUR_VERS_DUREE_IRRIGATION = 10

### Calcul du bilan hydrique

In [ ]:
# Choix de la culture
CULTURE = list(bilan.KC)[0]

# Choix du stade
STADE = list(bilan.KC[CULTURE])[0]

df_bilan = bilan.calcul_bilan(
    df_meteo_ref_si,
    TEXTURE, FRACTION_CAILLOUX,
    CULTURE, STADE,
    FRACTION_RU_REMPLIE, RU_VERS_RFU,
    seuil_irrigation=SEUIL_IRRIGATION,
    hauteur_vers_duree_irrigation=HAUTEUR_VERS_DUREE_IRRIGATION)

df_bilan.describe()